In [23]:
import uproot3 as uproot
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [2]:
degToRad=3.14159/180
polMagMap={
    "000":0.3519,
    "045":0.3374,
    "090":0.3303,
    "135":0.3375,
    "AMO":0.00001
}
polMagMap2={
    0:0.3519,
    45:0.3374,
    90:0.3303,
    135:0.3375,
    -1:0.00001
}
polAngMap={
    "000":0.0*degToRad,
    "045":45.0*degToRad,
    "090":90.0*degToRad,
    "135":135*degToRad,
    "AMO":0.0*degToRad
}
polMap={
    0:"000",
    45:"045",
    90:"090",
    135:"135",
    -1:"AMO"
}

variables=["event","cosTheta_eta_hel","phi_eta_hel","mandelstam_t","Mpi0eta","weightASBS","Phi","BeamAngle"]
mapNames={"event"}

In [5]:
#############################
# Load all the data
#############################

dataTags=["2018_8"] #["2017","2018_1","2018_8"]
mapDataset={}

fixedPolAngle=0 #None for all polarizations

for dataFormat in [""]:#["data","flat"]:
    print("NEXT FORMAT: "+dataFormat+"-----------------\n\n")
    datas=[]
    trees=[]
    dfs=[]

    totalEvents=0
    for i,dataTag in enumerate(dataTags):
        fileloc="/d/grid17/ln16/myDSelector/degALL_malte_kmatrix_2018_8_mEllipse_8288_tLT1_chi13_omegacut_treeFlat_DSelector.root"
        treename="tree_4g_flat"
#         fileloc="/d/grid17/ln16/myDSelector/degALL_flat_2018_8_mEllipse_8288_chi13_tLT1_omegacut_treeFlat_DSelector.root"
#         treename="tree_4g_flat"
        datas.append(uproot.open(fileloc))
        trees.append(datas[i][treename])
        dfs.append(trees[i].arrays(variables,outputtype=pandas.DataFrame))

        dfs[i].rename(
            columns={
            "event":"EventN",
            "cosTheta_eta_hel":"theta",
            "phi_eta_hel":"phi",
            "mandelstam_t":"tM",
            "Mpi0eta":"mass",
            "Phi":"alpha",
            "BeamAngle":"beamAngle"
            },
            inplace=True)

        # Adding some branches to the dataset
        dfs[i]["pol"]=dfs[0].beamAngle.map(polMagMap2)

        print("{0} has {1} events".format(dataTag,len(dfs[i])))
        totalEvents+=len(dfs[i])


    #############################
    # Split weights and data
    #############################
    # Fix some columns
    phase1_data=pandas.concat(dfs)
    print("total events {0}".format(totalEvents))
    phase1_data.theta=np.arccos(phase1_data.theta)
    phase1_data.phi=np.radians(phase1_data.phi)
    phase1_data.alpha=np.radians(phase1_data.alpha)
    phase1_data.tM = -1*phase1_data.tM
    
    # apply any cuts
    phase1_data=phase1_data[phase1_data.weightASBS!=0]
#     cut_tLT01GT03=(abs(phase1_data.tM)>0.1)&(abs(phase1_data.tM)<0.3)
#     phase1_data=phase1_data[cut_tLT01GT03]
    print("total events after some more selections {0}".format(len(phase1_data)))

    if fixedPolAngle is None:
        pols=[0,45,90,135,-1]
    else:
        pols=[0]
    
    for ipol,pol in enumerate(pols):
        # apply any final cuts
        if fixedPolAngle is None:
            phase1_data_subset = phase1_data[phase1_data.beamAngle==pol]
            phase1_data_subset.beamAngle = np.radians(phase1_data_subset.beamAngle)
        else:
            phase1_data_subset = phase1_data # do not cut if polarizations are fixed
            phase1_data_subset.beamAngle = np.radians(fixedPolAngle)
            
        print("After cuts (pol=={}): {}".format(pol,phase1_data_subset.shape))
        phase1_data_subset.reset_index(drop="true")

        # Split weight and data columns
        phase1_weight_subset=phase1_data_subset[["weightASBS","mass"]]
        phase1_data_subset=phase1_data_subset.drop("weightASBS",axis=1)
    
        #############################
        # Write to csvs
        #############################
    #     phase1_weight.to_csv("phase1_"+dataFormat+"_weights.csv",index=False)
    #     phase1_data.to_csv("phase1_"+dataFormat+"_data.csv",index=False)
        phase1_data_subset.to_csv("malte_kmatrix_data_"+polMap[pol]+".csv",index=False)
        phase1_weight_subset.to_csv("malte_kmatrix_weights_"+polMap[pol]+".csv",index=False)

NEXT FORMAT: -----------------


2018_8 has 607274 events
total events 607274
total events after some more selections 520587
After cuts (pol==0): (520587, 9)


In [19]:
# plt.hist(phase1_data_subset.reset_index(drop="true").mass,weights=phase1_weight_subset.reset_index(drop="true").weightASBS,bins=100)